## USE pretrained network to output keypoint's description

In [1]:
from __future__ import division, print_function
import glob
import os
import cv2
import PIL
import random
import numpy as np
# import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import torch
import torch.nn.init
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
import torch.nn.functional as F
import torchvision.datasets as dset
import torchvision.transforms as transforms
from tqdm import tqdm
from torch.autograd import Variable
from copy import deepcopy, copy
from config_profile import args
from Utils import cv2_scale36, cv2_scale, np_reshape, np_reshape64

In [2]:
from descriptor_CNN3 import DesNet
model = DesNet()
if args.cuda:
    model.cuda()

weight_path = "checkpoint.pth"
trained_weight = torch.load(weight_path)
model.load_state_dict(trained_weight['state_dict'])


In [3]:
# load patches
patches_dir_images = "../keypoint_detector/patches_images.pt"
patches_dir_query = "../keypoint_detector/patches_query.pt"
patches_dir_all = "../keypoint_detector/patches_all.pt"
patches_images = torch.load(patches_dir_images)
patches_query = torch.load(patches_dir_query)
patches_all = torch.load(patches_dir_all)

print(patches_images.shape)
print(patches_query.shape)
print(patches_all.shape)

patches_query =  patches_query.view(-1, 1, 32, 32).cuda()
patches_images =  patches_images.view(-1, 1, 32, 32).cuda()
patches_all =  patches_all.view(-1, 1, 32, 32).cuda()

torch.Size([140, 30, 1, 32, 32])
torch.Size([35, 30, 1, 32, 32])
torch.Size([175, 30, 1, 32, 32])


In [5]:
model.eval()
with torch.no_grad():
    description_images = model(patches_images)
    description_images = description_images.view(-1, 30, 128).cpu().data
    description_query = model(patches_query)
    description_query = description_query.view(-1, 30, 128).cpu().data
    description_all = model(patches_all)
    description_all = description_all.view(-1, 30, 128).cpu().data

    print(description_images.shape)
    print(description_query.shape)
    print(description_all.shape)


RuntimeError: CUDA out of memory. Tried to allocate 16.50 MiB (GPU 0; 10.92 GiB total capacity; 592.99 MiB already allocated; 14.50 MiB free; 783.00 KiB cached)

In [ ]:
## Save deep features  
# IMAGES
output_dir_images = "images_keypoints_descriptions.pt"
torch.save(description_images, output_dir_images)

# QUERY
output_dir_query = "query_keypoints_descriptions.pt"
torch.save(description_query, output_dir_query)

# QUERY + IMAGES
output_dir_query_and_images = "query_and_images_keypoints_descriptions.pt"
torch.save(description_all, output_dir_query_and_images)


In [35]:
# Load descriptions of the images
images_description = torch.load(output_dir_images)
query_description = torch.load(output_dir_query)
query_and_images_description = torch.load(output_dir_query_and_images)

print(query_description.shape)
print(images_description.shape)
print(query_and_images_description.shape)

#print(query_description)
#print(query_and_images_description)
#print(images_description)


torch.Size([35, 30, 128])
torch.Size([140, 30, 128])
torch.Size([175, 30, 128])


In [36]:
# One-to-one keypoint matching: Compute the cost matrix


In [37]:
from munkres import Munkres

sim_matrix = np.zeros((35,140))

for qkeypoint in range(35):
    for images_keypoint in range(140):
        cost_matrix = np.zeros((30, 30))
        for i in range(30):
            for j in range(30):
                cost_matrix[i][j] = np.linalg.norm(query_description[qkeypoint][i].cpu().numpy() - images_description[images_keypoint][j].cpu().numpy())
        # Hungarian: one-to-one matching
        m = Munkres()
        indexes = m.compute(np.copy(cost_matrix))
        
        for m in indexes:
                sim_matrix[qkeypoint, images_keypoint] += np.exp(-cost_matrix[m])


In [55]:
print(sim_matrix)
#np.savetxt("one_to_one_similitude_matrix", sim_matrix, delimiter=",")
one_to_one_similitude_tensor = torch.as_tensor(sim_matrix)

print(one_to_one_similitude_tensor.shape)
print(one_to_one_similitude_tensor)
torch.save(one_to_one_similitude_tensor, "one_to_one_similitude_matrix.pt")
test = torch.load("one_to_one_similitude_matrix.pt")
print(test.shape)
print(test)


[[16.09582365 15.95460675 15.27731852 ...  8.89354472  8.96293825
   8.82521085]
 [ 9.52089546  9.51881673  9.47644828 ...  9.50603487  9.55114158
   9.23158531]
 [ 9.74975131  9.79641903  9.62664098 ...  9.076581    9.04206852
   9.21711886]
 ...
 [ 8.86574528  8.90869303  8.75763617 ...  9.09690092  9.06312987
   9.38135474]
 [ 8.96228402  9.13603875  8.76421999 ...  9.02715686  9.07895744
   9.10371407]
 [ 9.13884786  8.9311002   8.84456485 ... 10.18426631 17.81534871
  11.70028291]]
torch.Size([35, 140])
tensor([[16.0958, 15.9546, 15.2773,  ...,  8.8935,  8.9629,  8.8252],
        [ 9.5209,  9.5188,  9.4764,  ...,  9.5060,  9.5511,  9.2316],
        [ 9.7498,  9.7964,  9.6266,  ...,  9.0766,  9.0421,  9.2171],
        ...,
        [ 8.8657,  8.9087,  8.7576,  ...,  9.0969,  9.0631,  9.3814],
        [ 8.9623,  9.1360,  8.7642,  ...,  9.0272,  9.0790,  9.1037],
        [ 9.1388,  8.9311,  8.8446,  ..., 10.1843, 17.8153, 11.7003]],
       dtype=torch.float64)
torch.Size([35, 140])
te

In [26]:
# Many to many matching

In [56]:
sim_matrix_many_to_many = np.zeros((35,140))

for qkeypoint in range(35):
    for images_keypoint in range(140):
        cost_matrix = np.zeros((30, 30))
        for i in range(30):
            for j in range(30):
                cost_matrix[i][j] = np.linalg.norm(query_description[qkeypoint][i].cpu().numpy() - images_description[images_keypoint][j].cpu().numpy())

        sim_matrix = np.exp(-cost_matrix)
        x = sim_matrix/ np.linalg.norm(sim_matrix, axis=0)
        sim_matrix_many_to_many[qkeypoint][images_keypoint] = np.multiply(sim_matrix, x).sum()
        
     
        

In [35]:
#np.savetxt("many_to_many_similitude_matrix", sim_matrix_many_to_many, delimiter=",")

In [58]:
#print(sim_matrix_many_to_many[1])
print(sim_matrix_many_to_many)
#np.savetxt("one_to_one_similitude_matrix", sim_matrix, delimiter=",")
sim_matrix_many_to_many_tensor = torch.as_tensor(sim_matrix_many_to_many)

print(sim_matrix_many_to_many_tensor.shape)
print(sim_matrix_many_to_many_tensor)
torch.save(sim_matrix_many_to_many_tensor, "many_to_many_similitude_matrix.pt")
test2 = torch.load("many_to_many_similitude_matrix.pt")
print(test2.shape)
print(test2)
 

[[51.7252415  50.62483538 51.34427346 ... 44.00673066 43.55393212
  43.21341456]
 [46.51604753 46.30991434 46.43907204 ... 47.23570939 46.66816236
  45.44350613]
 [45.06540521 45.20175599 45.25944013 ... 44.84995313 44.51627118
  44.35296605]
 ...
 [43.30583658 43.58107569 43.37359059 ... 44.64019902 44.25281825
  44.79963042]
 [44.18130976 44.43467499 43.74936925 ... 44.90966726 44.62806266
  44.80982697]
 [44.51699584 43.99887962 43.8592617  ... 48.92385104 52.4003462
  47.40526241]]
torch.Size([35, 140])
tensor([[51.7252, 50.6248, 51.3443,  ..., 44.0067, 43.5539, 43.2134],
        [46.5160, 46.3099, 46.4391,  ..., 47.2357, 46.6682, 45.4435],
        [45.0654, 45.2018, 45.2594,  ..., 44.8500, 44.5163, 44.3530],
        ...,
        [43.3058, 43.5811, 43.3736,  ..., 44.6402, 44.2528, 44.7996],
        [44.1813, 44.4347, 43.7494,  ..., 44.9097, 44.6281, 44.8098],
        [44.5170, 43.9989, 43.8593,  ..., 48.9239, 52.4003, 47.4053]],
       dtype=torch.float64)
torch.Size([35, 140])
ten

In [7]:
# Evaluate keypoint matching by estimating precision and recall of image retrieval

print(one_to_one_similitude_matrix.shape)
queryIDs, indexes_sim_matrix = np.loadtxt("../image_retrieval/ground_truth.txt", dtype={'names': ['label', 'age'],'formats': ('U10', 'i4')}, usecols =(0, 1), unpack = True)

print(queryIDs.size)
print(queryIDs)
print(indexes_sim_matrix.size)
print(indexes_sim_matrix)

torch.Size([35, 140])
140
['q1' 'q1' 'q1' 'q1' 'q2' 'q2' 'q2' 'q2' 'q3' 'q3' 'q3' 'q3' 'q4' 'q4'
 'q4' 'q4' 'q5' 'q5' 'q5' 'q5' 'q6' 'q6' 'q6' 'q6' 'q7' 'q7' 'q7' 'q7'
 'q8' 'q8' 'q8' 'q8' 'q9' 'q9' 'q9' 'q9' 'q10' 'q10' 'q10' 'q10' 'q11'
 'q11' 'q11' 'q11' 'q12' 'q12' 'q12' 'q12' 'q13' 'q13' 'q13' 'q13' 'q14'
 'q14' 'q14' 'q14' 'q15' 'q15' 'q15' 'q15' 'q16' 'q16' 'q16' 'q16' 'q17'
 'q17' 'q17' 'q17' 'q18' 'q18' 'q18' 'q18' 'q19' 'q19' 'q19' 'q19' 'q20'
 'q20' 'q20' 'q20' 'q21' 'q21' 'q21' 'q21' 'q22' 'q22' 'q22' 'q22' 'q23'
 'q23' 'q23' 'q23' 'q24' 'q24' 'q24' 'q24' 'q25' 'q25' 'q25' 'q25' 'q26'
 'q26' 'q26' 'q26' 'q27' 'q27' 'q27' 'q27' 'q28' 'q28' 'q28' 'q28' 'q29'
 'q29' 'q29' 'q29' 'q30' 'q30' 'q30' 'q30' 'q31' 'q31' 'q31' 'q31' 'q32'
 'q32' 'q32' 'q32' 'q33' 'q33' 'q33' 'q33' 'q34' 'q34' 'q34' 'q34' 'q35'
 'q35' 'q35' 'q35']
140
[  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  3

In [107]:
one_to_one_similitude_matrix = torch.load("results/one_to_one_similitude_matrix.pt")
many_to_many_similitude_matrix = torch.load("results/many_to_many_similitude_matrix.pt")

# K = most similar images
K = 2
K_one_to_one = np.zeros((35,K))
K_many_to_many = np.zeros((35,K))
#print(K)
id = 0
for i in range(queryIDs.size):
    #print(queryIDs[i])
    for queryID in range(36):
        #id = 0
        if queryIDs[i] == 'q'+ str(queryID):
           # print("hello " + 'q'+  str(queryID))
            m = indexes_sim_matrix[i]
           # print(m)
            sim_one_to_one = one_to_one_similitude_matrix[queryID-1][i].data.cpu().numpy()
            sim_many_to_many = many_to_many_similitude_matrix[queryID-1][i].data.cpu().numpy()
            #print(sim)
            K_one_to_one[queryID-1][id] = sim_one_to_one
            K_many_to_many[queryID-1][id] = sim_many_to_many
            id += 1
        if id == K:
            id = 0
            
#print(K_one_to_one.size)
#print(K_one_to_one)

#print(K_many_to_many.size)
#print(K_many_to_many)

In [108]:
# Determine true or false positives

In [30]:
def Nmaxelements(list1, N): 
    l = list1.copy()
    final_list = []
  
    for i in range(0, N):  
        max1 = 0
          
        for j in range(len(list1)):      
            if list1[j] > max1: 
                max1 = list1[j];
                list1[j] = 0;
               # print(l.index(max1))
                  
        #list1.remove(max1); 
        #final_list[l.index(max1)+1] = max1 
        final_list.append(max1)
          
    #print(final_list) 
    return final_list

In [109]:
one_to_one_similitude_matrix = torch.load("results/one_to_one_similitude_matrix.pt")
one_to_one_similitude_matrix = one_to_one_similitude_matrix.numpy()

highest_matching_points = np.zeros((35,K))
true_positives = np.zeros((35,K))
precision_matrix = np.zeros((35,1))

for i in range(35):
        highest_matching_points[i] = Nmaxelements(one_to_one_similitude_matrix[i], K)
      


    
for i in range(35):
    K_one_to_one[i].sort();
    highest_matching_points[i].sort()
    total_true_positives = 0
    for j in range(K):
        if K_one_to_one[i][j] == highest_matching_points[i][j]:
            true_positives[i][j] = 1.
            total_true_positives += 1
            #print(K_one_to_one[i][j])
        else:
            true_positives[i][j] = 0.
            #print(K_one_to_one[i][j])
            #print(highest_matching_points[i][j])
    precision_matrix[i] = total_true_positives/K

    
#print(precision_matrix)
average_precision = 0
for i in range(35):
    average_precision += precision_matrix[i]

average_precision = average_precision / 35
print("AVERAGE", average_precision)
#print(K_one_to_one)
#print(highest_matching_points)
#print(true_positives)
#print(total_true_positives)
#precision = total_true_positives / K

#print(precision)
        

AVERAGE [0.3]


In [110]:
many_to_many_similitude_matrix = torch.load("results/many_to_many_similitude_matrix.pt")
many_to_many_similitude_matrix = many_to_many_similitude_matrix.numpy()

highest_matching_points_many_to_many = np.zeros((35,K))
true_positives_many_to_many = np.zeros((35,K))
precision_matrix_many_to_many = np.zeros((35,1))

for i in range(35):
        highest_matching_points_many_to_many[i] = Nmaxelements(many_to_many_similitude_matrix[i], K)
      


    
for i in range(35):
    K_many_to_many[i].sort();
    highest_matching_points_many_to_many[i].sort()
    total_true_positives_many_to_many = 0
    for j in range(K):
        if K_many_to_many[i][j] == highest_matching_points_many_to_many[i][j]:
            true_positives_many_to_many[i][j] = 1.
            total_true_positives_many_to_many += 1
            #print(K_one_to_one[i][j])
        else:
            true_positives_many_to_many[i][j] = 0.
            #print(K_one_to_one[i][j])
            #print(highest_matching_points[i][j])
    precision_matrix_many_to_many[i] = total_true_positives_many_to_many/K

    
#print(precision_matrix_many_to_many)
average_precision = 0
for i in range(35):
    average_precision += precision_matrix_many_to_many[i]

average_precision = average_precision / 35
print("PRECISION AVERAGE", average_precision)
#print(K_one_to_one)
#print(highest_matching_points)
#print(true_positives_many_to_many)
#print(total_true_positives_many_to_many)
#precision = total_true_positives / K

#print(precision)

PRECISION AVERAGE [0.22857143]


In [ ]:
# Determine precision and recall